## 1. Importing and exploring the dataset

### Importing the required libraries 

In [1]:
import pandas as pd 
import numpy as np
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

### Opening the file and working with only boroughs that contain the word Toronto 

In [2]:
path = "c:/Bhavin/Git/DS_capstone/Coursera_Capstone/neighborhood.csv"

df = pd.read_csv(path)
df.drop("Unnamed: 0",axis=1,inplace=True)


for i in range(df.shape[0]):
    borough = df.loc[i,"Borough"]
    if "Toronto" not in borough:
        df.drop(i,axis=0,inplace=True)

df.reset_index(inplace=True)
df.drop("index",axis=1,inplace=True)
df.head()

PostalCode           Borough                                 Neighborhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.676357 -79.293031

#### Finding out the different number of boroughs and neighbourhoods

In [3]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


### Creating the Map highlighting all the locations

In [4]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 2. Exploring the Neighborhoods in Toronto

#### Defining Foursquare version and ID

In [5]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200404' # Foursquare API version

### Getting the top 50 Venues of each neighbourhood

In [6]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
#     print(venues_list)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    tempdata = [item for venue_list in venues_list for item in venue_list]
#     print(tempdata)
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [8]:
LIMIT = 50

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
# df.head()

#### Checking how many venues were returned for each neighborhood

In [9]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            50   
Brockton, Parkdale Village, Exhibition Place                           24   
Business reply mail Processing Centre                                  17   
CN Tower, King and Spadina, Railway Lands, Harb...                     15   
Central Bay Street                                                     50   
Christie                                                               17   
Church and Wellesley                                                   50   
Commerce Court, Victoria Hotel                                         50   
Davisville                                                             35   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           16   
First Canadian Place, Underground city                                 50   
Forest Hill North & West                                                4   
Garden District, Ryerson                                               50   
Harbourfront East, Union Station, Toronto Islands                      50   
High Park, The Junction South                                          23   
India Bazaar, The Beaches West                                         19   
Kensington Market, Chinatown, Grange Park                              50   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               43   
Moore Park, Summerhill East                                             2   
North Toronto West                                                     21   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            33   
Regent Park, Harbourfront                                              48   
Richmond, Adelaide, King                                               50   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede, Swansea                                                     35   
St. James Town                                                         50   
St. James Town, Cabbagetown                                            44   
Stn A PO Boxes                                                         50   
Studio District                                                        41   
Summerhill West, Rathnelly, South Hill, Forest ...                     16   
The Annex, North Midtown, Yorkville                                    23   
The Beaches                                                             4   
The Danforth West, Riverdale                                           42   
Toronto Dominion Centre, Design Exchange                               50   
University of Toronto, Harbord                                         35   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             50   
Brockton, Parkdale Village, Exhibition Place                            24   
Business reply mail Processing Centre                                   17   
CN Tower, King and Spadina, Railway Lands, Harb...                      15   
Central Bay Street                                                      50   
Christie                                                                17   
Church and Wellesley                                                    50   
Commerce Court, Victoria Hotel                                          50   
Davisville                                                              35   
Davisville North                    

#### Finding out how many unique categories can be curated from all the returned venues

In [10]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 221 uniques categories.


## 3. Analyzing Each Neighborhood

In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.tail()

Yoga Studio  Airport  Airport Food Court  Airport Gate  Airport Lounge  \
1182            0        0                   0             0               0   
1183            0        0                   0             0               0   
1184            0        0                   0             0               0   
1185            0        0                   0             0               0   
1186            0        0                   0             0               0   

      Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
1182                0                 0                    0             0   
1183                0                 0                    0             0   
1184                0                 0                    0             0   
1185                0                 0                    0             0   
1186                0                 0                    0             0   

      Aquarium  ...  Theater  Theme Restaurant  Toy / Game Store  Trail  \
1182         0  ...        0                 0                 0      0   
1183         0  ...        0                 0                 0      0   
1184         0  ...        0                 0                 0      0   
1185         0  ...        0                 0                 0      0   
1186         0  ...        0                 0                 0      0   

      Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
1182              0                              0                 0   
1183              0                              0                 0   
1184              0                              0                 0   
1185              0                              0                 0   
1186              0                              0                 0   

      Vietnamese Restaurant  Wine Bar  Wings Joint  
1182                      0         0            0  
1183                      0         0            0  
1184                      0         0            0  
1185                      0         0            0  
1186                      0         0            0  

[5 rows x 221 columns]

Examining the new dataframe size.

In [12]:
toronto_onehot.shape

(1187, 221)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio   Airport  \
0                                        Berczy Park     0.000000  0.000000   
1       Brockton, Parkdale Village, Exhibition Place     0.041667  0.000000   
2              Business reply mail Processing Centre     0.000000  0.000000   
3  CN Tower, King and Spadina, Railway Lands, Har...     0.000000  0.066667   
4                                 Central Bay Street     0.020000  0.000000   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0            0.000000      0.000000        0.000000              0.0   
1            0.000000      0.000000        0.000000              0.0   
2            0.000000      0.000000        0.000000              0.0   
3            0.066667      0.066667        0.066667              0.2   
4            0.000000      0.000000        0.000000              0.0   

   Airport Terminal  American Restaurant  Antique Shop  ...  Theater  \
0          0.000000                  0.0           0.0  ...      0.0   
1          0.000000                  0.0           0.0  ...      0.0   
2          0.000000                  0.0           0.0  ...      0.0   
3          0.133333                  0.0           0.0  ...      0.0   
4          0.000000                  0.0           0.0  ...      0.0   

   Theme Restaurant  Toy / Game Store  Trail  Train Station  \
0               0.0               0.0    0.0            0.0   
1               0.0               0.0    0.0            0.0   
2               0.0               0.0    0.0            0.0   
3               0.0               0.0    0.0            0.0   
4               0.0               0.0    0.0            0.0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                           0.02               0.0                    0.0   
1                           0.00               0.0                    0.0   
2                           0.00               0.0                    0.0   
3                           0.00               0.0                    0.0   
4                           0.02               0.0                    0.0   

   Wine Bar  Wings Joint  
0       0.0          0.0  
1       0.0          0.0  
2       0.0          0.0  
3       0.0          0.0  
4       0.0          0.0  

[5 rows x 221 columns]

#### Let's print each neighborhood along with the top 5 most common venues

In [14]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2                Café  0.04
3            Beer Bar  0.04
4  Seafood Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1  Breakfast Spot  0.08
2     Coffee Shop  0.08
3     Yoga Studio  0.04
4    Intersection  0.04


----Business reply mail Processing Centre----
                  venue  freq
0    Light Rail Station  0.12
1            Smoke Shop  0.06
2         Garden Center  0.06
3               Brewery  0.06
4  Fast Food Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.20
1     Airport Terminal  0.13
2      Harbor / Marina  0.07
3              Airport  0.07
4  Rental Car Location  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1        

####  Creating a new dataframe to display the top 10 venues for each neighborhood.

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2              Business reply mail Processing Centre    Light Rail Station   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar                Bakery    Seafood Restaurant   
1        Breakfast Spot           Coffee Shop           Yoga Studio   
2            Skate Park         Burrito Place  Fast Food Restaurant   
3      Airport Terminal               Airport    Airport Food Court   
4                  Café    Italian Restaurant        Sandwich Place   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0             Restaurant           Cheese Shop              Beer Bar   
1  Performing Arts Venue                   Bar         Burrito Place   
2         Farmers Market         Auto Workshop                   Spa   
3           Airport Gate        Airport Lounge              Boutique   
4        Bubble Tea Shop          Burger Joint        Ice Cream Shop   

  8th Most Common Venue      9th Most Common Venue 10th Most Common Venue  
0                  Café             Clothing Store                  Hotel  
1            Restaurant               Climbing Gym              Nightclub  
2            Smoke Shop                    Brewery             Restaurant  
3         Boat or Ferry        Rental Car Location                  Plane  
4           Yoga Studio  Middle Eastern Restaurant       Ramen Restaurant

## 4. Cluster Neighborhoods

Running K-means to cluster the neighborhoods into 5-Cluster

In [17]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 



array([4, 4, 4, 4, 4, 4, 4, 4, 4, 1])

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!





PostalCode           Borough                                 Neighborhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.654260 -79.360636               4           Coffee Shop   
1  43.662301 -79.389494               4           Coffee Shop   
2  43.657162 -79.378937               4                  Café   
3  43.651494 -79.375418               4                  Café   
4  43.676357 -79.293031               0                 Trail   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Bakery                   Pub                  Park   
1      Sushi Restaurant           Wings Joint                 Diner   
2              Tea Room      Ramen Restaurant            Restaurant   
3             Gastropub           Coffee Shop   American Restaurant   
4     Health Food Store                   Pub           Wings Joint   

  5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
0        Breakfast Spot            Restaurant                         Café   
1                  Park    Mexican Restaurant          Japanese Restaurant   
2             Bookstore               Theater               Cosmetics Shop   
3        Cosmetics Shop              Creperie                 Cocktail Bar   
4          Dance Studio     Electronics Store  Eastern European Restaurant   

       8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                    Theater           Yoga Studio     Mexican Restaurant  
1         Italian Restaurant            Hobby Shop                    Gym  
2  Middle Eastern Restaurant        Clothing Store            Coffee Shop  
3         Seafood Restaurant    Italian Restaurant         Farmers Market  
4                 Donut Shop      Doner Restaurant                Dog Run

Visualizing the clusters


In [19]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters